In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [2]:
# Verify the number of reviews that were read (100,000 in total)
haha = train["review"].size; hihi = test["review"].size
hoho = unlabeled_train["review"].size
print(f"Read {haha} labeled train reviews, {hihi} labeled test reviews, and {hoho} unlabeled reviews\n")
#print(f"Read {train["review"].size} labeled train reviews, {hihi} labeled test reviews, and {hoho} unlabeled reviews\n")

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#No need to run the below line again after it has been run 
#nltk.download()
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eric\anaconda3\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "..." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:414: MarkupResemblesLocatorWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


Parsing sentences from unlabeled set


C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:414: MarkupResemblesLocatorWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:414: MarkupResemblesLocatorWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:329: MarkupResemblesLocatorWarning: "... ..." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Eric\anaconda3\lib\site-packages\bs4\__init__.py:3

In [7]:
print(len(sentences))

795538


In [8]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [9]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [10]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-08-18 18:42:29,893 : INFO : collecting all words and their counts
2020-08-18 18:42:29,894 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-18 18:42:29,937 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2020-08-18 18:42:29,982 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2020-08-18 18:42:30,029 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2020-08-18 18:42:30,076 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2020-08-18 18:42:30,132 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2020-08-18 18:42:30,173 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2020-08-18 18:42:30,230 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2020-08-18 18:42:30,282 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2020-08-18 18:42:30,328 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2020-08-18 18:42:30,379 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2020-08-18 18:42:30,423 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2020-08-18 18:42:30,468 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2020-08-18 18:42:33,540 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2020-08-18 18:42:33,585 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2020-08-18 18:42:33,632 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2020-08-18 18:42:33,678 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2020-08-18 18:42:33,703 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2020-08-18 18:42:33,704 : INFO : Loading a fresh vocabulary
2020-08-18 18:42:34,495 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2020-08-18 18:42:34,496 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2020-08-18 18:42:34,547 : INFO : deleting the raw counts dictionary of 123504 items
2020-08-18 18:42:34,551 : I

2020-08-18 18:43:25,710 : INFO : EPOCH 4 - PROGRESS: at 49.21% examples, 776313 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:26,714 : INFO : EPOCH 4 - PROGRESS: at 56.88% examples, 798407 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:27,714 : INFO : EPOCH 4 - PROGRESS: at 64.79% examples, 819911 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:28,720 : INFO : EPOCH 4 - PROGRESS: at 70.10% examples, 806715 words/s, in_qsize 8, out_qsize 0
2020-08-18 18:43:29,725 : INFO : EPOCH 4 - PROGRESS: at 74.68% examples, 788267 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:30,733 : INFO : EPOCH 4 - PROGRESS: at 78.84% examples, 767931 words/s, in_qsize 8, out_qsize 1
2020-08-18 18:43:31,736 : INFO : EPOCH 4 - PROGRESS: at 84.78% examples, 767078 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:32,742 : INFO : EPOCH 4 - PROGRESS: at 92.68% examples, 782736 words/s, in_qsize 7, out_qsize 0
2020-08-18 18:43:33,639 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-

In [11]:
model.doesnt_match("man woman child kitchen".split())

<ipython-input-11-c4c7e8da36b4>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  model.doesnt_match("man woman child kitchen".split())
C:\Users\Eric\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [12]:
model.doesnt_match("paris berlin london austria".split())

<ipython-input-12-45844e2ac28d>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  model.doesnt_match("paris berlin london austria".split())


'paris'

In [13]:
model.most_similar("man")

<ipython-input-13-216414787a03>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("man")


[('woman', 0.6304882168769836),
 ('lad', 0.5952304005622864),
 ('lady', 0.5862138867378235),
 ('soldier', 0.5298150777816772),
 ('guy', 0.5144491195678711),
 ('men', 0.5124490857124329),
 ('farmer', 0.5104303359985352),
 ('chap', 0.5053516626358032),
 ('poet', 0.5001262426376343),
 ('monk', 0.4985868036746979)]

In [14]:
model.most_similar("asian")

<ipython-input-14-631ed7059b1a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("asian")


[('indian', 0.6352517604827881),
 ('italian', 0.6323878765106201),
 ('hispanic', 0.6223514080047607),
 ('chinese', 0.6198261976242065),
 ('european', 0.602661669254303),
 ('japanese', 0.580132246017456),
 ('american', 0.5754120945930481),
 ('african', 0.5719794034957886),
 ('afro', 0.5574480891227722),
 ('korean', 0.5480692386627197)]

In [15]:
model.most_similar("awful")

<ipython-input-15-efd9e43fbdfc>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("awful")


[('terrible', 0.7755510807037354),
 ('horrible', 0.7380619049072266),
 ('atrocious', 0.7331347465515137),
 ('dreadful', 0.7232122421264648),
 ('abysmal', 0.7059292793273926),
 ('appalling', 0.6836127042770386),
 ('horrendous', 0.6766362190246582),
 ('horrid', 0.676304280757904),
 ('lousy', 0.6320223808288574),
 ('amateurish', 0.6204963326454163)]

In [16]:
haha = model.wv['awful']
haha, haha.size

(array([-0.13040787, -0.0387306 , -0.02428757,  0.03284191,  0.00774023,
        -0.02209255,  0.1190803 ,  0.15986356,  0.06376628, -0.01328486,
         0.0607137 , -0.06253612,  0.01057846,  0.0115566 ,  0.12447701,
        -0.05760406, -0.15813698, -0.07051598,  0.03741488, -0.05992971,
         0.02909673,  0.0772497 , -0.02052126,  0.01426938,  0.01180677,
         0.06953558,  0.08850285, -0.07516508,  0.03543423, -0.00883125,
        -0.0226506 , -0.05088933, -0.0665429 , -0.0685086 , -0.01138698,
         0.04999544,  0.02665504, -0.0590856 , -0.06763619, -0.0189209 ,
        -0.02010874, -0.04241288,  0.08553794,  0.09579083, -0.11264968,
         0.00942093,  0.06050405, -0.04189688, -0.010866  , -0.03346559,
         0.01149091, -0.00743175,  0.02160575, -0.02055809,  0.04755779,
        -0.01132907,  0.02819855,  0.03973909,  0.02506342, -0.04406499,
         0.04346847, -0.07829822,  0.0120181 , -0.02365987,  0.08024968,
        -0.0107654 , -0.02055994,  0.04025637, -0.0

In [17]:
#part 3 started here
#changes below were made according to 
#https://groups.google.com/g/gensim/c/hlYgjqEVocw
#some methods and attributes were moved to
#KeyedVectors class from word2vec class
from gensim.models import Word2Vec, KeyedVectors

In [18]:
model = KeyedVectors.load("300features_40minwords_10context")

2020-08-18 18:43:52,170 : INFO : loading Word2VecKeyedVectors object from 300features_40minwords_10context
2020-08-18 18:43:52,466 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2020-08-18 18:43:52,467 : INFO : setting ignored attribute vectors_norm to None
2020-08-18 18:43:52,468 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2020-08-18 18:43:52,468 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2020-08-18 18:43:52,468 : INFO : setting ignored attribute cum_table to None
2020-08-18 18:43:52,469 : INFO : loaded 300features_40minwords_10context


In [19]:
#changed: wv. added
type(model.wv.syn0)

<ipython-input-19-04073011a372>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  type(model.wv.syn0)


numpy.ndarray

In [20]:
model.wv.syn0.shape

<ipython-input-20-b77a3a51ff8b>:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  model.wv.syn0.shape


(16490, 300)

In [21]:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)
all_stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'did',
 'didn',
 'do',
 'does',
 'doesn',
 'doing',
 'don',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifte

In [22]:
#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set and word in index2word_set2: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000 == 0:
            haha = counter; hihi = len(reviews)
            print(f"Review {haha} of {hihi}") #% (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        #reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [23]:
index2word_set = set(model.wv.index2word)
index2word_set2 = all_stopwords

haha = []
for hihi in index2word_set2:
    if hihi not in index2word_set:
        haha.append(hihi)
print(haha), print(index2word_set2)

['noone', 'couldnt', 'ltd', 'hereupon', 'hereafter', 'amoungst', 'whither', 'thence', 'whereafter', 'whereupon', 'whence', 'thereupon', 'kg', 'hereby', 'hasnt', 'whereby', 'latterly', 'km']
{'us', 'again', 'beside', 'there', 'once', 'often', 'had', 'same', 'even', 'forty', 'seeming', 'somewhere', 'more', 'than', 'whether', 'although', 'over', 'one', 'after', 'above', 'found', 'if', 'do', 'sometimes', 'becomes', 'keep', 'computer', 'them', 'otherwise', 'since', 'enough', 'everything', 'ie', 'before', 'nevertheless', 'describe', 'please', 'other', 'it', 'hence', 'amongst', 'co', 'am', 'noone', 'each', 'at', 'into', 'couldnt', 'make', 'him', 'that', 'whole', 'name', 'afterwards', 'and', 'amount', 'therefore', 'front', 'hers', 'top', 'he', 'last', 'while', 'out', 'elsewhere', 'but', 'neither', 'see', 'been', 'fifteen', 'be', 'seem', 'anyone', 'therein', 'you', 'of', 'upon', 'made', 'an', 'now', 'no', 'together', 'formerly', 'somehow', 'here', 'whereas', 'perhaps', 'because', 'meanwhile', '

(None, None)

In [24]:
type(model)

gensim.models.word2vec.Word2Vec

In [25]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True ))
    clean_train_reviews.append( review_to_wordlist( review ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    #clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    clean_test_reviews.append( review_to_wordlist( review ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


<ipython-input-22-d4aced638c63>:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec,model[word])


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [36]:
type(train)

pandas.core.frame.DataFrame

In [37]:
type(trainDataVecs)

numpy.ndarray

In [26]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [27]:
print(output)

               id  sentiment
0      "12311_10"          1
1        "8348_2"          0
2        "5828_4"          1
3        "7186_2"          0
4       "12128_7"          1
...           ...        ...
24995   "2155_10"          0
24996     "59_10"          1
24997    "2531_1"          0
24998    "7772_8"          1
24999  "11465_10"          1

[25000 rows x 2 columns]


In [33]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids